In [1]:
import spotipy
import os
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd

In [2]:
client_id = "706f63df6ce2464a84798d54a4188fda" 
client_secret = "927e4b99ecd243c0a30d178084d0d4ac"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

df = pd.concat(pd.read_excel('inputs/initial_songs.xlsx', sheet_name=None), ignore_index=True)

In [3]:
def extract_features(URL, genre):
    genre = genre
    playlist_id = URL
    
    playlist_tracks_data = sp.playlist_tracks(playlist_id)
    tracks = playlist_tracks_data['items']
    while playlist_tracks_data['next']:
        playlist_tracks_data = sp.next(playlist_tracks_data)
        tracks.extend(playlist_tracks_data['items'])
        
    playlist_tracks_id = []
    playlist_tracks_titles = []
    features = []

    for track in tracks:
        if(track['track']['id']!=None):
            playlist_tracks_id.append(track['track']['id'])
            playlist_tracks_titles.append(track['track']['name'])
    
    for i in range(0,len(playlist_tracks_id),50):
        #spotipy for whatever reason can only handle 50 with audio_features
        audio_features = sp.audio_features(playlist_tracks_id[i:i+50])
        
        for track in audio_features:
            features.append(track)
    
    df = pd.DataFrame(features)
    df['genre'] = genre
    df['title'] = playlist_tracks_titles
    return df

In [4]:
def create_df(playlist_csv):
    result = pd.DataFrame()
    
    # Extract songs from spotify API and append to dataframe
    for i in range(len(playlist_csv)):
        df = extract_features(playlist_csv['playlistid'][i], playlist_csv['genre'][i])
        result = pd.concat([result, df], ignore_index=True)
        
    return result

In [5]:
songs = create_df(df)

In [6]:
songs.to_csv('outputs/edm_songs.csv')

In [7]:
validation_df = pd.concat(pd.read_excel('inputs/val_set_songs.xlsx', sheet_name=None), ignore_index=True)
validation_songs = create_df(validation_df)
validation_songs.to_csv('outputs/validation_songs.csv')